In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain, SequentialChain

# Configurar el modelo OpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Paso 1 - Prompt de generación con Chain of Thought
system_template = "Eres un escritor creativo que razona paso a paso."
human_template = (
    "Piensa en pasos lógicos y escribe un texto de 5 líneas "
    "sobre '{topic}' en estilo {style}."
)

system_msg_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_msg_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_msg_prompt, human_msg_prompt])

generate_chain = LLMChain(llm=llm, prompt=chat_prompt, output_key="generated_text")

# Paso 2 - Refinar el texto generado
refine_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Ahora eres un editor que pule textos."),
    HumanMessagePromptTemplate.from_template(
        "Pulir el siguiente texto para mejorar claridad y estilo:\n\n{generated_text}"
    )
])
refine_chain = LLMChain(llm=llm, prompt=refine_prompt, output_key="refined_text")

# Cadena secuencial
chain = SequentialChain(
    chains=[generate_chain, refine_chain],
    input_variables=["topic", "style"],
    output_variables=["generated_text", "refined_text"],
    verbose=True,
)

# Ejecutar el chain
outputs = chain({"topic": "la luna", "style": "poético y evocativo"})
print("Texto generado:\n", outputs["generated_text"])
print("\nTexto refinado:\n", outputs["refined_text"])


/tmp/ipykernel_18898/2738910266.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
/tmp/ipykernel_18898/2738910266.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  generate_chain = LLMChain(llm=llm, prompt=chat_prompt, output_key="generated_text")
/tmp/ipykernel_18898/2738910266.py:40: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  outputs = chain({"topic": "la luna", "style": "poético y evocativo"})




> Entering new SequentialChain chain...

> Finished chain.
Texto generado:
 En el lienzo nocturno, la luna asoma,  
un faro plateado que susurra secretos,  
acariciando las olas con su luz de antaño,  
dibujando sombras danzantes en la tierra,  
y recordándonos que en la oscuridad, hay siempre un brillo que guía.

Texto refinado:
 En el lienzo de la noche, la luna asoma,  
un faro plateado que susurra secretos,  
acariciando las olas con su luz ancestral,  
dibujando sombras danzantes en la tierra  
y recordándonos que, en la oscuridad, siempre hay un brillo que nos guía.
